In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.utils import to_categorical
import glob

# Set your parameters
img_height, img_width = 128, 128
num_classes = 93  # Change this to the number of classes you have

# Create a mapping of labels to numeric indices
def create_label_mapping(data_dir):
    labels = set()
    for img_path in glob.glob(os.path.join(data_dir, '*.png')):
        label = os.path.basename(img_path).split('-')[0]
        labels.add(label)
    return {label: idx for idx, label in enumerate(sorted(labels))}

# Load images and labels
def load_data(data_dir, label_mapping):
    images = []
    labels = []
    
    for img_path in glob.glob(os.path.join(data_dir, '*.png')):
        # Read the image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        img = cv2.resize(img, (img_width, img_height))  # Resize to match the input shape
        images.append(img)
        
        # Extract the label from the filename
        label = os.path.basename(img_path).split('-')[0]  # Modify this based on your naming convention
        labels.append(label_mapping[label])  # Use the mapping to get the integer label
    
    images = np.array(images)
    images = images.astype('float32') / 255.0  # Normalize the images
    labels = np.array(labels)

    return images, labels

# Load the dataset
data_dir = 'Dataset/dataset/'  # Path to your dataset folder
label_mapping = create_label_mapping(data_dir)  # Create the label mapping
X, y = load_data(data_dir, label_mapping)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for LSTM input
X_train = X_train.reshape((X_train.shape[0], 1, img_height, img_width))  # Add time dimension
X_val = X_val.reshape((X_val.shape[0], 1, img_height, img_width))  # Add time dimension

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

# Define the model
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None, img_height, img_width, 1)),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
    TimeDistributed(Flatten()),
    LSTM(100, return_sequences=False),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=5,
    verbose=1
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')


2024-10-24 09:00:50.488672: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 09:00:50.638903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 09:00:50.699981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 09:00:50.717125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 09:00:50.820548: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 91s 214ms/step - accuracy: 0.0900 - loss: 4.1550 - val_accuracy: 0.1065 - val_loss: 4.0148
Epoch 2/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 91s 217ms/step - accuracy: 0.0909 - loss: 4.0444 - val_accuracy: 0.1065 - val_loss: 4.0092
Epoch 3/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 90s 216ms/step - accuracy: 0.0909 - loss: 4.0502 - val_accuracy: 0.1065 - val_loss: 4.0106
Epoch 4/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 92s 219ms/step - accuracy: 0.0904 - loss: 4.0571 - val_accuracy: 0.1065 - val_loss: 4.0093
Epoch 5/5
419/419 ━━━━━━━━━━━━━━━━━━━━ 90s 216ms/step - accuracy: 0.0947 - loss: 4.0417 - val_accuracy: 0.1065 - val_loss: 4.0140
Validation Loss: 4.0140, Validation Accuracy: 0.1065


In [5]:
# Save the model
model.save('saved_model/my_model.keras')  # Save the model in HDF5 format

In [6]:
from tensorflow.keras.models import load_model

# Function to load the model and test it on new images
def test_model(model_path, test_data_dir, label_mapping):
    # Load the saved model
    model = load_model(model_path)
    
    # Load test images
    test_images, test_labels = load_data(test_data_dir, label_mapping)  # You can modify this function as needed for test images

    # Reshape test images for LSTM
    test_images = test_images.reshape((test_images.shape[0], 1, img_height, img_width))

    # Make predictions
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)

    return predicted_classes, test_labels

# Test the model
test_data_dir = 'Dataset/test/'  # Change this to your test dataset path
predicted_classes, actual_labels = test_model('saved_model/my_model.keras', test_data_dir, label_mapping)

# Output the predictions
for i in range(len(predicted_classes)):
    print(f'Test Image {i}: Predicted Class {predicted_classes[i]}, Actual Class {actual_labels[i]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Test Image 0: Predicted Class 38, Actual Class 0
Test Image 1: Predicted Class 38, Actual Class 0
Test Image 2: Predicted Class 38, Actual Class 0
Test Image 3: Predicted Class 38, Actual Class 0
Test Image 4: Predicted Class 38, Actual Class 0
Test Image 5: Predicted Class 38, Actual Class 0
Test Image 6: Predicted Class 38, Actual Class 0
Test Image 7: Predicted Class 38, Actual Class 0
Test Image 8: Predicted Class 38, Actual Class 0
Test Image 9: Predicted Class 38, Actual Class 0
Test Image 10: Predicted Class 38, Actual Class 0


In [16]:
# Model architecture
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Dropout

input_image = Input(shape=(128, 32, 1), name='image_input')
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_image)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Reshape(target_shape=(32, -1))(x)  # Adjust shape for LSTM layers
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)

output = Dense(len(char_to_num) + 1, activation='softmax')(x)  # CTC needs a blank token

# Compile the Model using CTC Loss
model = Model(inputs=input_image, outputs=output)
model.compile(optimizer='adam', loss=ctc_loss)
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 128, 32, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 128, 32, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 64, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 64, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 32, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 8, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 32, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_10                │ (None, 32, 256)        │       656,384 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ (None, 32, 256)        │       394,240 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32, 80)         │        20,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,090,384 (4.16 MB)

 Trainable params: 1,090,192 (4.16 MB)

 Non-trainable params: 192 (768.00 B)

In [18]:

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=50,
    verbose=1
)




Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/ctc_loss/boolean_mask/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/bhavin/miniconda3/envs/Try/lib/python3.12/asyncio/base_events.py", line 1986, in _run_once

  File "/home/bhavin/miniconda3/envs/Try/lib/python

In [ ]:
# Save the trained model
model.save('handwriting_recognition_model.h5')

# Convert to TensorFlow Lite for mobile
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('handwriting_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)